In [1]:
import ctypes

In [2]:
basic_dll = ctypes.CDLL('fourier_basic.so')

In [3]:
basic_dll.density_fourier_capi_float.restype = ctypes.c_int
basic_dll.density_fourier_capi_float.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_long, ctypes.c_long, ctypes.c_float, ctypes.c_float]
basic_dll.density_fourier_capi_double.restype = ctypes.c_int
basic_dll.density_fourier_capi_double.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_long, ctypes.c_long, ctypes.c_double, ctypes.c_double]

In [4]:
import numpy as np

In [5]:
def density_fourier_float(data: np.array, hcount = 32):
    fp_data = np.ascontiguousarray(data, dtype = np.float32)
    scount = len(fp_data)
    re_harm, im_harm = np.zeros(hcount, dtype = np.float32), np.zeros(hcount, dtype = np.float32) 
    dmin, dmax = np.min(data), np.max(data)
    shift, basek = 0.5 * (dmax + dmin), 2 * np.pi / np.abs(dmax - dmin)
    res = basic_dll.density_fourier_capi_float(  \
        fp_data.ctypes.data_as(ctypes.c_void_p), \
        re_harm.ctypes.data_as(ctypes.c_void_p), \
        im_harm.ctypes.data_as(ctypes.c_void_p), \
        scount, hcount, shift, basek)
    assert res == 0
    return (re_harm, im_harm)

def density_fourier_double(data: np.array, hcount = 32):
    fp_data = np.ascontiguousarray(data, dtype = np.float64)
    scount = len(fp_data)
    re_harm, im_harm = np.zeros(hcount, dtype = np.float64), np.zeros(hcount, dtype = np.float64) 
    dmin, dmax = np.min(data), np.max(data)
    shift, basek = 0.5 * (dmax + dmin), 2 * np.pi / np.abs(dmax - dmin)
    res = basic_dll.density_fourier_capi_double( \
        fp_data.ctypes.data_as(ctypes.c_void_p), \
        re_harm.ctypes.data_as(ctypes.c_void_p), \
        im_harm.ctypes.data_as(ctypes.c_void_p), \
        scount, hcount, shift, basek)
    assert res == 0
    return (re_harm, im_harm)

def density_fourier(data: np.array, hcount = 32):
    if data.dtype == np.float32:
        return density_fourier_float(data, hcount)
    if data.dtype == np.float64:
        return density_fourier_double(data, hcount)
    return None

In [6]:
import matplotlib.pyplot as plt

In [7]:
basic_dll.evaluate_fourier_capi_float.restype = ctypes.c_float
basic_dll.evaluate_fourier_capi_float.argtypes = [ctypes.c_float, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_long, ctypes.c_float, ctypes.c_float]
basic_dll.evaluate_fourier_capi_double.restype = ctypes.c_double
basic_dll.evaluate_fourier_capi_double.argtypes = [ctypes.c_double, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_long, ctypes.c_double, ctypes.c_double]

In [8]:
def evaluate_fourier_float(arg: float, 
                           reharmonics: np.array, imharmonics: np.array, 
                           shift = 0.0, basek = np.pi) -> float:
    assert (imharmonics.ndim == 1) and (reharmonics.ndim == 1)
    assert imharmonics.shape == reharmonics.shape
    reh = np.ascontiguousarray(reharmonics, dtype = np.float32)
    imh = np.ascontiguousarray(imharmonics, dtype = np.float32)
    hcount = len(imh)
    return basic_dll.evaluate_fourier_capi_float(     \
                arg,                                  \
                reh.ctypes.data_as(ctypes.c_void_p),  \
                imh.ctypes.data_as(ctypes.c_void_p),  \
                hcount, shift, basek) / reharmonics[0]

def evaluate_fourier_double(arg: float, 
                            reharmonics: np.array, imharmonics: np.array, 
                            shift = 0.0, basek = np.pi) -> float:
    assert (imharmonics.ndim == 1) and (reharmonics.ndim == 1)
    assert imharmonics.shape == reharmonics.shape
    reh = np.ascontiguousarray(reharmonics, dtype = np.float64)
    imh = np.ascontiguousarray(imharmonics, dtype = np.float64)
    hcount = len(imh)
    return basic_dll.evaluate_fourier_capi_double(   \
                arg,                                  \
                reh.ctypes.data_as(ctypes.c_void_p),  \
                imh.ctypes.data_as(ctypes.c_void_p),  \
                hcount, shift, basek) / reharmonics[0]

def evaluate_fourier(arg: float, 
                     reharmonics: np.array, imharmonics: np.array, 
                     shift = 0.0, basek = np.pi):
    assert imharmonics.dtype == reharmonics.dtype
    if (imharmonics.dtype == np.float32) and (reharmonics.dtype == np.float32):
        return evaluate_fourier_float(arg, reharmonics, imharmonics, shift, basek)
    if (imharmonics.dtype == np.float64) and (reharmonics.dtype == np.float64):
        return evaluate_fourier_double(arg, reharmonics, imharmonics, shift, basek)
    return None

In [9]:
import pandas as pd

In [ ]:
higgs = pd.read_csv('../data/HIGGS.csv.gz', header = None)

In [ ]:
higgs.to_pickle('../data/higgs.pickle')

In [10]:
higgs = pd.read_pickle('../data/higgs.pickle')

In [ ]:
plt.figure(figsize = (7.5, 10), dpi = 120)
for c in [2, 7, 11, 15, 19]:
    reex, imex = density_fourier(higgs[c].to_numpy())
    xs = np.linspace(-1.0, +1.0, 1000)
    func = lambda x: evaluate_fourier(x, reex, imex)
    ys = np.array([func(x) for x in xs])
    plt.plot(xs, ys, label = f'Column = {c}')
plt.legend()

In [11]:
train_size = 10000000
train, test = higgs.iloc[:train_size, :], higgs.iloc[train_size:, :]
del higgs

In [12]:
train_true, train_false = train[train[0] == 1.0], train[train[0] == 0.0]

In [13]:
train_true_data = train_true.iloc[:, 1:]
train_false_data = train_false.iloc[:, 1:]

In [14]:
nfeatures = train_true_data.shape[1]
nharmonics = 32
hshape = (nharmonics, nfeatures)
reharmonics, imharmonics = np.zeros(hshape), np.zeros(hshape)

In [15]:
for f in range(nfeatures):
    feature = train_true_data.iloc[:, f].to_numpy()
    r, i = density_fourier(feature, hcount = nharmonics)
    reharmonics[:, f], imharmonics[:, f] = r, i

In [16]:
from numba import *

In [17]:
@jit(nopython = True)
def ev(x, fn, reh, imh):
    return evaluate_fourier(x, reh[:, fn], imh[:, fn])

@jit(nopython = True, parallel = True)
def handle_pos(row, nf, errmat, reh, imh):
    for f1 in range(nf):
        v1 = ev(r[f1], f1, reh, imh)
        for f2 in range(nf):
            v2 = ev(r[f2], f2, reh, imh)
            err = 1 - v1 * v2
            errmat[f1, f2] += (err * err)
            
@jit(nopython = True, parallel = True)
def handle_neg(row, errmat):
    for f1 in range(nf):
        v1 = ev(r[f1], f1, reh, imh)
        for f2 in range(nf):
            v2 = ev(r[f2], f2, reh, imh)
            err = v1 * v2
            errmat[f1, f2] += (err * err)

In [18]:
error_matrix = np.zeros((nfeatures, nfeatures))
for i, row in train_true_data.iterrows():
    handle_pos(row.to_numpy(), nfeatures, error_matrix, reharmonics, imharmonics)
for i, r in train_false_data.iterrows():
    handle_neg(row.to_numpy(), nfeatures, error_matrix, reharmonics, imharmonics)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'evaluate_fourier': cannot determine Numba type of <class 'function'>

File "<ipython-input-17-b7f30c3f8062>", line 3:
def ev(x, fn, reh, imh):
    return evaluate_fourier(x, reh[:, fn], imh[:, fn])
    ^

During: resolving callee type: type(CPUDispatcher(<function ev at 0x1110dfdc0>))
During: typing of call at <ipython-input-17-b7f30c3f8062> (8)

During: resolving callee type: type(CPUDispatcher(<function ev at 0x1110dfdc0>))
During: typing of call at <ipython-input-17-b7f30c3f8062> (10)

During: resolving callee type: type(CPUDispatcher(<function ev at 0x1110dfdc0>))
During: typing of call at <ipython-input-17-b7f30c3f8062> (8)


File "<ipython-input-17-b7f30c3f8062>", line 8:
def handle_pos(row, nf, errmat, reh, imh):
    <source elided>
    for f1 in range(nf):
        v1 = ev(r[f1], f1, reh, imh)
        ^
